<a href="https://colab.research.google.com/github/hwizard-wph/NAS/blob/main/Reinforce_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x

!curl -O https://storage.googleapis.com/nasbench/nasbench_full.tfrecord

!git clone https://github.com/google-research/nasbench
!pip install ./nasbench

from nasbench import api

nasbench = api.NASBench('nasbench_full.tfrecord')

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.distributions import Categorical

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

INPUT = 'input'
OUTPUT = 'output'
CONV3X3 = 'conv3x3-bn-relu'
CONV1X1 = 'conv1x1-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'
NUM_VERTICES = 7
MAX_EDGES = 9
EDGE_SPOTS = NUM_VERTICES * (NUM_VERTICES - 1) / 2
OP_SPOTS = NUM_VERTICES - 2 
ALLOWED_OPS = [CONV3X3, CONV1X1, MAXPOOL3X3]
ALLOWED_EDGES = [0, 1] 

In [ ]:
def get_spec(genotypes):
  matrix = np.zero(NUM_VERTICES, NUM_VERTICES)
  ops = []
  


In [ ]:
class Policy(nn.Module):
  def __init__(self):
    self.max_nodes = NUM_VERTICES
    self.search_space = ALLOWED_OPS
    self.edge2index = {}
    for i in range(1, self.max_nodes):
      for j in range(i):
        node_str = "{:}<-{:}".format(i, j)
        self.edge2index[node_str] = len(self.edge2index)
    self.arch_parameters = nn.Parameter(1e-3 * torch.randn(len(self.edge2index), len(self.search_space)))
  
  def generate_arch(self, action):
    genotypes = []
    for i in range(1, self.max_nodes):
        xlist = []
        for j in range(i):
            node_str = "{:}<-{:}".format(i, j)
            op_name = self.search_space[actions[self.edge2index[node_str]]]
            xlist.append((op_name, j))
        genotypes.append(tuple(xlist))
    return get_spec(genotypes)